### NLP: comprehension exercises for statistical reasoning.

Matthew Stone   
Initial version, Spring 2018 -- revised for Python 3 Spring 2019.  


[Resampling for approximate permutation tests.][1]

[1]:https://en.wikipedia.org/wiki/Resampling_(statistics)

Rather than find an appropriate test statistic and interpret it according to a set procedure, it's often easier to simply simulate the results of your experiment assuming the null hypothesis (that is, assuming that any effects you observed were simply due to chance). You can use the fraction of time the simulation gets results similar to yours as an estimate of the  *p*-value associated with your experimental results. This is technically known as an approximate permutation test in statistics.

This exercise has two goals.
- Understand [the code below](#code) and appreciate the reasoning behind it.
- Get a feel for the kinds of results it gives: on the one hand, small data sets make it surprisingly difficult to detect even fairly substantial differences in performance; on the other hand, extremely large data sets can establish that even trivial differences are reliable.

There are five questions to fill out:
- [Setup some sample variables](#setup)
- [Look at the variability in performance of learners across small experiments](#var1)
- [Relate this to your ability as an experimenter to trust experimental results](#var2)
- [Quantify this using the approximate randomization test](#sig)
- [Reflect on your results](#reflect)


### Code
<a id="code"/>

Here, the null hypothesis is to assume that there's no difference in overall accuracy between the results of two classifiers. That means it doesn't really matter which classifier made which prediction; if we swap the classifiers randomly, we'll get similar results. Here sim_expt proposes to swap items from p1 and p2 at random and see whether the resulting simulated experiment passes the passed test. The test we'll be interested in is whether the difference in accuracy between the "two algorithms" exceeds the difference we observed in the actual experiment. 


In [1]:
import random
import numpy as np
import sklearn
import sklearn.metrics

def sim_expt(test, p1, p2):
    '''apply test to a simulated experiment 
       where the difference between p1 and p2 is randomly erased'''
    bits = [random.getrandbits(1) for i in range(0, len(p1))]
    return test([p1[i] if bits[i] else p2[i] for i in range(0,len(p1))],
                [p2[i] if bits[i] else p1[i] for i in range(0,len(p1))])

def eval_diff(data, p1, p2) :
    '''a test generator based on the performance difference between p1 and p2
       asks whether the result of a simulated experiment is at least as extreme
       as the observed accuracy difference between p1 and p2 on data'''
    diff = abs(sklearn.metrics.accuracy_score(data, p1) -
               sklearn.metrics.accuracy_score(data, p2))
    def test_diff(n, m) :
        return (abs(sklearn.metrics.accuracy_score(data, n) - 
                    sklearn.metrics.accuracy_score(data, m)) >= 
                diff)
    return test_diff

def mcmcp_diff(data, p1, p2, k) :
    '''the approximate permuatation test:
       simulate an experiment like that used to obtain p1 and p2
       forgetting any difference between p1 and p2
       running the experiment k times
       and record the probability that the performance difference on data
       is as extreme as actually observed'''
    success = 0
    test = eval_diff(data, p1, p2)
    for i in range(0,k) :
        success += sim_expt(test, p1, p2)
    return success / float(k)

###  Setup

<a id="setup"/>

Define `data` to contain 100K uniformly random bits: `data` will represent the true class of a bunch of items in your test set.

Suppose you have an algorithm *A1* that gets 80% accuracy on the true distribution from which `data` is drawn.  If you draw a data point at random, and don't know what it is, then you can "simulate" the effect of classifying the data point with *A1* by a random choice.  Using this idea, simulate using *A1* on all of `data`, and store the results in `p1`

Similarly, simulate the results of another algorithm *A2* on `data`, assuming *A2* classifies points with accuracy 83%.  Store the results in `p2`.

### Variability, I
<a id="var1"/>

You can think of `data`, `p1`, and `p2` as
- 1 big experiment of size 100K: E1
- 10 experiments of size 10K: E10
- 100 experiments of size 1K: E100
- 1K experiments of size 100: E1000

It's easy to see that the accuracy of *A1* and *A2* in E1 has to be equal to the average accuracy across all the other experiments.  In the cell below, write an expression that calculates (and displays) the overall accuracy of *A1* and *A2*.  

It's more interesting to consider the [standard deviations](https://en.wikipedia.org/wiki/Standard_deviation) of the accuracy of *A1* and *A2* across experiments E10, E100, and E1000.  In the cell below, write an expression that calculates (and displays) those standard deviations.  [Let me save you a google search for numpy standard deviation.](#https://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html)

### Variability, II
<a id="var2"/>

Another way of asking a similar question: in how many of the experiments in E10, E100 and E1000, does *A1* (misleadingly) wind up with better performance than *A2*?  Asking this question is the essence of statistical thinking.  

In the cell below, write python expressions that compute the answers for E10, E100 and E1000.

### Significance
<a id="sig"/>

Pick any one experiment in each of E10, E100, and E1000 (your choice, random is good).
For each one, write code in the cell below to compute the following statistics.
- Measure the observed difference in performance between *A1* and *A2* on this experiment.
- Use the approximate permutation test [code](#code) to compute the chance probability of a difference as large as what you see, according to the approximate permutation test, with k=2500?  (You are trading off accuracy versus waiting with this number.)

### Conclusion
<a id="reflect"/>

In the markdown cell below, give an intuitive summary and explanation of the results of your notebook.